## Question generation and rewriting for product domain

In [ ]:
!pip install openai
!pip install instructor
!pip install pydantic

In [ ]:
import base64
import os
import json
from openai import OpenAI
from instructor import from_openai
from pydantic import BaseModel
from typing import List

# OpenAI API Key
api_key = ""
client = from_openai(OpenAI(api_key=api_key))
model = "gpt-4o"


class SinglePair(BaseModel):
    direct: str
    indirect: str


class Response(BaseModel):
    response: List[SinglePair]


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def get_message_for_images(base64_image):
    messages = [
        {
            "role": "system",
            "content": """
                I have an image of a product that contains specific details, such as labels, branding, or visible features. This product could be anything that has a label and can be bought from platforms like Amazon, such as electronics, household items, beauty products, etc.
                Your task is to generate a set of at least 20 unique indirect question/command pairs related to the product.
                Each pair should consist of a direct question/command referring to the product in general terms and an indirect counterpart that mentions the product explicitly by name or specific feature.
                Ensure the questions or commands cover various domains, such as searching, checking prices, availability, purchase options, reviews, sharing, product details, and product comparison.
                The output should be strictly formatted in JSON with a list of 20 dictionaries with 'direct' and 'indirect' as keys and the respective questions or commands as values.
                """,
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": ""},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{base64_image}"},
                },
            ],
        },
    ]
    return messages


def get_direct_indirect_questions(image_path):
    # Getting the base64 string
    base64_image = encode_image(image_path)

    messages = get_message_for_images(base64_image=base64_image)
    response = client.chat.completions.create(
        model=model, response_model=Response, messages=messages
    )

    return response.model_dump(mode="json")


# Main logic to process images in a folder and save results to a JSON file
folder_path = "D:/product_images"  # Specify the folder containing the images
output_file = "results.json"  # Specify the output file name

results = []

# Iterate through all the images in the folder
for image_file in os.listdir(folder_path):
    if not image_file.endswith((".png", ".jpg", ".jpeg")):
        continue  # Skip non-image files

    image_path = os.path.join(folder_path, image_file)
    image_id = os.path.splitext(image_file)[0]

    try:
        # Get direct/indirect question pairs
        direct_indirect_questions = get_direct_indirect_questions(image_path)

        # Loop through each pair in the response
        for entry in direct_indirect_questions["response"]:
            prompt = f"Rewrite this: {entry['indirect']}"
            response = entry["direct"]

            results.append(
                {"ImageId": image_id, "Prompt": prompt, "RewrittenQuestion": response}
            )
    except Exception as e:
        print(f"Query failed for image {image_id}. Error: {str(e)}. Retrying...")
        continue


# Save results to JSON
with open(output_file, "w") as outfile:
    json.dump(results, outfile, indent=4)